In [5]:
import torch
import numpy as np
import cv2
device = 'cuda:0'

In [6]:
def rotate_yaw(x, y, z, rot):
    xx = x * np.cos(rot) - y * np.sin(rot)
    yy = x * np.sin(rot) + y * np.cos(rot)
    zz = z

    return xx, yy, zz

def rotate_roll(x, y, z, rot):
    xx = x
    yy = y * np.cos(rot) - z * np.sin(rot)
    zz = y * np.sin(rot) + z * np.cos(rot)

    return xx, yy, zz

def rotate_pitch(x, y, z, rot):
    xx = x * np.cos(rot) + z * np.sin(rot)
    yy = y
    zz = -x * np.sin(rot) + z * np.cos(rot)

    return xx, yy, zz

In [7]:
import cv2
import numpy as np

# 画像の読み込み
img = cv2.imread("./data/data_100/Room/0/O.png")
h, w = img.shape[:2]

w_half = int(w / 2)
h_half = int(h / 2)

# メッシュグリッドの生成
phi, theta = np.meshgrid(np.linspace(-np.pi, np.pi, w_half*2),
                         np.linspace(-np.pi/2, np.pi/2, h_half*2))


# 球面座標
x = np.cos(theta) * np.cos(phi)
y = np.cos(theta) * np.sin(phi)
z = np.sin(theta)

# z軸周りにπ/2ラジアン回転
rad = np.pi / 2
xx, yy, zz = rotate_yaw(x, y, z, rad)
xx, yy, zz = rotate_pitch(xx, yy, zz, rad)
xx, yy, zz = rotate_yaw(xx, yy, zz, rad)

# 逆球面座標変換
theta = np.arcsin(zz)
phi = np.arctan2(yy, xx)

# 画像座標へのマッピング
Y = 2*theta / np.pi * h_half + h_half
X = phi / np.pi * w_half + w_half

# 画像のリマッピングと保存
out = cv2.remap(img, X.astype(np.float32), Y.astype(np.float32), cv2.INTER_LINEAR, borderMode=cv2.BORDER_WRAP)
cv2.imwrite("dst.png", out)


True

In [8]:
def spherical_to_cartesian(phi, theta):
    x = np.cos(theta) * np.cos(phi)
    y = np.cos(theta) * np.sin(phi)
    z = np.sin(theta)
    return x, y, z

def rotate_coordinates(x, y, z, angle=np.pi/2):
    xx = x * np.cos(angle) + z * np.sin(angle)
    yy = y
    zz = -x * np.sin(angle) + z * np.cos(angle)
    return xx, yy, zz

def cartesian_to_spherical(x, y, z):
    theta = np.arcsin(z)
    phi = np.arctan2(y, x)
    return phi, theta

In [9]:
def convert_coordinate(input_xy, image_size_hw):
    h, w = image_size_hw
    w_half = w / 2
    h_half = h / 2
    
    phi = (input_xy[0] - w_half) * np.pi * 2 /w
    theta = (input_xy[1] - h_half) * np.pi /h
    
    x, y, z = spherical_to_cartesian(phi, theta)
    rad = -np.pi / 2
    xx, yy, zz = rotate_yaw(x, y, z, rad)
    xx, yy, zz = rotate_pitch(xx, yy, zz, rad)
    xx, yy, zz = rotate_yaw(xx, yy, zz, rad)
    new_phi, new_theta = cartesian_to_spherical(xx, yy, zz)

    new_y = 2*new_theta * h_half / np.pi + h_half
    new_x = new_phi * w_half / np.pi + w_half
    
    return new_x, new_y

In [15]:
img = cv2.imread("./data/data_100/Room/0/O2.png")
image_size = (512, 1024)

# 点の位置
position = (784, 368)
position3 = (255, 400)
# 点の色（BGR形式）
color = (0, 0, 255)  # 緑色

# 点を描画
cv2.drawMarker(img, position, color, markerType=cv2.MARKER_CROSS, markerSize=40, thickness=2, line_type=cv2.LINE_AA)
cv2.drawMarker(img, position3, color, markerType=cv2.MARKER_CROSS, markerSize=40, thickness=2, line_type=cv2.LINE_AA)
cv2.imshow('Image with a marker', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [14]:
img2 = cv2.imread("./data/data_100/Room/0/O.png")

# 点の位置
position2 = convert_coordinate(position, image_size)
position2 = (int(position2[0]), int(position2[1]))
position32 = convert_coordinate(position3, image_size)
position32 = (int(position32[0]), int(position32[1]))

# 点の色（BGR形式）
color = (0, 0, 255)  # 緑色

# 点を描画
cv2.drawMarker(img2, position2, color, markerType=cv2.MARKER_CROSS, markerSize=40, thickness=2, line_type=cv2.LINE_AA)
cv2.drawMarker(img2, position32, color, markerType=cv2.MARKER_CROSS, markerSize=40, thickness=2, line_type=cv2.LINE_AA)
cv2.imshow('Image with a marker', img2)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [21]:
print(position2)

(748, 399)


In [130]:
img = cv2.imread("./O.png")
h, w = img.shape[:2]

w_half = int(w / 2)
h_half = int(h / 2)

# メッシュグリッドの生成
phi, theta = np.meshgrid(np.linspace(-np.pi, np.pi, w_half*2),
                         np.linspace(-np.pi/2, np.pi/2, h_half*2))


# 球面座標
x = np.cos(theta) * np.cos(phi)
y = np.cos(theta) * np.sin(phi)
z = np.sin(theta)

# z軸周りにπ/2ラジアン回転
rot = np.pi / 2
xx = x * np.cos(rot) + z * np.sin(rot)
yy = y
zz = -x * np.sin(rot) + z * np.cos(rot)

# 逆球面座標変換
theta = np.arcsin(zz)
phi = np.arctan2(yy, xx)

# 画像座標へのマッピング
Y = 2*theta / np.pi * h_half + h_half
X = phi / np.pi * w_half + w_half

# 画像のリマッピングと保存
out = cv2.remap(img, X.astype(np.float32), Y.astype(np.float32), cv2.INTER_LINEAR, borderMode=cv2.BORDER_WRAP)
cv2.imwrite("dst.png", out)

True